In [0]:
# Importing dependencies numpy and keras
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
# load text
! wget https://raw.githubusercontent.com/cs109/2015/master/Lectures/Lecture15b/sparklect/shakes/macbeth.txt
filename = "macbeth.txt"

text = (open(filename).read()).lower()

# mapping characters with integers
unique_chars = sorted(list(set(text)))

char_to_int = {}
int_to_char = {}

for i, c in enumerate (unique_chars):
    char_to_int.update({c: i})
    int_to_char.update({i: c})

--2019-05-03 17:03:10--  https://raw.githubusercontent.com/cs109/2015/master/Lectures/Lecture15b/sparklect/shakes/macbeth.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119097 (116K) [text/plain]
Saving to: ‘macbeth.txt’

macbeth.txt         100%[===================>] 116.31K  --.-KB/s    in 0.03s   

2019-05-03 17:03:11 (3.78 MB/s) - ‘macbeth.txt’ saved [119097/119097]



In [0]:
# preparing input and output dataset
X = []
Y = []

for i in range(0, len(text) - 50, 1):
    sequence = text[i:i + 50]
    label =text[i + 50]
    X.append([char_to_int[char] for char in sequence])
    Y.append(char_to_int[label])

In [0]:
X

[[49,
  11,
  14,
  10,
  14,
  0,
  42,
  30,
  27,
  1,
  42,
  40,
  23,
  29,
  27,
  26,
  47,
  1,
  37,
  28,
  1,
  35,
  23,
  25,
  24,
  27,
  42,
  30,
  0,
  0,
  0,
  24,
  47,
  1,
  45,
  31,
  34,
  34,
  31,
  23,
  35,
  1,
  41,
  30,
  23,
  33,
  27,
  41,
  38,
  27],
 [11,
  14,
  10,
  14,
  0,
  42,
  30,
  27,
  1,
  42,
  40,
  23,
  29,
  27,
  26,
  47,
  1,
  37,
  28,
  1,
  35,
  23,
  25,
  24,
  27,
  42,
  30,
  0,
  0,
  0,
  24,
  47,
  1,
  45,
  31,
  34,
  34,
  31,
  23,
  35,
  1,
  41,
  30,
  23,
  33,
  27,
  41,
  38,
  27,
  23],
 [14,
  10,
  14,
  0,
  42,
  30,
  27,
  1,
  42,
  40,
  23,
  29,
  27,
  26,
  47,
  1,
  37,
  28,
  1,
  35,
  23,
  25,
  24,
  27,
  42,
  30,
  0,
  0,
  0,
  24,
  47,
  1,
  45,
  31,
  34,
  34,
  31,
  23,
  35,
  1,
  41,
  30,
  23,
  33,
  27,
  41,
  38,
  27,
  23,
  40],
 [10,
  14,
  0,
  42,
  30,
  27,
  1,
  42,
  40,
  23,
  29,
  27,
  26,
  47,
  1,
  37,
  28,
  1,
  35,
  23,
  25,
  

In [0]:
Y

[23,
 40,
 27,
 0,
 0,
 0,
 0,
 26,
 40,
 23,
 35,
 23,
 42,
 31,
 41,
 1,
 38,
 27,
 40,
 41,
 37,
 36,
 23,
 27,
 0,
 0,
 1,
 1,
 26,
 43,
 36,
 25,
 23,
 36,
 7,
 1,
 33,
 31,
 36,
 29,
 1,
 37,
 28,
 1,
 41,
 25,
 37,
 42,
 34,
 23,
 36,
 26,
 0,
 1,
 1,
 35,
 23,
 25,
 24,
 27,
 42,
 30,
 7,
 1,
 42,
 30,
 23,
 36,
 27,
 1,
 37,
 28,
 1,
 29,
 34,
 23,
 35,
 31,
 41,
 1,
 23,
 36,
 26,
 1,
 25,
 23,
 45,
 26,
 37,
 40,
 7,
 1,
 23,
 1,
 29,
 27,
 36,
 27,
 40,
 23,
 34,
 1,
 31,
 36,
 1,
 42,
 30,
 27,
 1,
 33,
 31,
 36,
 29,
 4,
 41,
 0,
 23,
 40,
 35,
 47,
 0,
 1,
 1,
 34,
 23,
 26,
 47,
 1,
 35,
 23,
 25,
 24,
 27,
 42,
 30,
 7,
 1,
 30,
 31,
 41,
 1,
 45,
 31,
 28,
 27,
 0,
 1,
 1,
 35,
 23,
 25,
 26,
 43,
 28,
 28,
 7,
 1,
 42,
 30,
 23,
 36,
 27,
 1,
 37,
 28,
 1,
 28,
 31,
 28,
 27,
 7,
 1,
 23,
 1,
 36,
 37,
 24,
 34,
 27,
 35,
 23,
 36,
 1,
 37,
 28,
 1,
 41,
 25,
 37,
 42,
 34,
 23,
 36,
 26,
 0,
 1,
 1,
 34,
 23,
 26,
 47,
 1,
 35,
 23,
 25,
 26,
 43,
 28,
 28,
 7,
 1,


In [0]:
# reshaping, normalizing and one hot encoding
X_modified = numpy.reshape(X, (len(X), 50, 1))
X_modified = X_modified / float(len(unique_chars))
Y_modified = np_utils.to_categorical(Y)

In [0]:
# defining the LSTM model
model = Sequential()
model.add(LSTM(300, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# fitting the model
model.fit(X_modified, Y_modified, epochs=1, batch_size=30)

# picking a random seed
start_index = numpy.random.randint(0, len(X)-1)
new_string = X[start_index]

# generating characters
for i in range(50):
    x = numpy.reshape(new_string, (1, len(new_string), 1))
    x = x / float(len(unique_chars))

    #predicting
    pred_index = numpy.argmax(model.predict(x, verbose=0))
    char_out = int_to_char[pred_index]
    seq_in = [int_to_char[value] for value in new_string]
    print(char_out)

    new_string.append(pred_index)
    new_string = new_string[1:len(new_string)]

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
116089/116089 [==============================] - 599s 5ms/step - loss: 2.7831
 
 
 
 
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
t
h
e
 
 
 
 
 
t


In [0]:
# fitting the model
history = model.fit(X_modified, Y_modified, epochs=10, batch_size=30)

# picking a random seed
start_index = numpy.random.randint(0, len(X)-1)
new_string = X[start_index]

# generating characters
for i in range(50):
    x = numpy.reshape(new_string, (1, len(new_string), 1))
    x = x / float(len(unique_chars))

    #predicting
    pred_index = numpy.argmax(model.predict(x, verbose=0))
    char_out = int_to_char[pred_index]
    seq_in = [int_to_char[value] for value in new_string]
    print(char_out)

    new_string.append(pred_index)
    new_string = new_string[1:len(new_string)]

Epoch 1/10
116089/116089 [==============================] - 588s 5ms/step - loss: 2.4297
Epoch 2/10
116089/116089 [==============================] - 584s 5ms/step - loss: 2.3015
Epoch 3/10
116089/116089 [==============================] - 581s 5ms/step - loss: 2.2059
Epoch 4/10
116089/116089 [==============================] - 703s 6ms/step - loss: 2.1208
Epoch 5/10
116089/116089 [==============================] - 585s 5ms/step - loss: 2.0504
Epoch 6/10
 92730/116089 [======================>.......] - ETA: 2:03 - loss: 1.9929